In [1]:
import numpy as np
import pandas as pd

# my slice stats

In [154]:
dataFrameStats = pd.DataFrame.from_csv("D:/Dropbox/DFCI/PY-LUNG/output/Lung1_SliceStats.csv", index_col = None)
# delete first column
del dataFrameStats['Unnamed: 0']
print dataFrameStats.shape
dataFrameStats.head(n=5)

(537, 36)


,patient,nodule,Xspacing,Yspacing,Zspacing,Xmin,Ymin,Zmin,Xcount,Ycount,...,maskYmax_G,xPixelRange_G,yPixelRange_G,maskXmin_B,maskXmax_B,maskYmin_B,maskYmax_B,xPixelRange_B,yPixelRange_B,smoothed
0,43062,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-166.0,512.0,512.0,...,247.0,96.0,72.0,129.0,225.0,175.0,247.0,96.0,72.0,True
1,43370,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-320.5,512.0,512.0,...,323.0,62.0,44.0,285.0,317.0,288.0,317.0,32.0,29.0,True
2,43392,0,0.976562,0.976562,3.0,-249.511719,-249.511719,-169.5,512.0,512.0,...,96.0,63.0,31.0,295.0,335.0,71.0,99.0,40.0,28.0,True
3,43402,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-136.0,512.0,512.0,...,142.0,71.0,68.0,171.0,217.0,74.0,142.0,46.0,68.0,True
4,43421,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-227.5,512.0,512.0,...,247.0,59.0,66.0,275.0,335.0,181.0,247.0,60.0,66.0,True


# clinical

In [71]:
dataFrameClinical = pd.DataFrame.from_csv("D:/Dropbox/DFCI/04_data/Lung1 clinical/LungAllPatients2011_04_20.csv", index_col = None)
dataFrameClinical.head(n=5)


,idstudy,patis,name,Birthdte,birthday,birthmon,birthyea,gender,intake_who,nicotuse,...,height_fatandgrip,fat_perc_baseline,gripleft_baseline,gripright_baseline,prefside_hand,Date_meas_RT,weight_RT,fat_perc_RT,gripleft_RT,gripright_RT
0,,97,Loo,1/1/1943,1,1,1943,2,1,2,...,162,27,20,19,right-handed,1/18/2010,54,29.2000007629395,23,17
1,,530,Wirtz,3/1/1955,1,3,1955,2,,,...,,,,,,,,,,
2,787,636,Spijkers,3/6/1943,6,3,1943,2,1,1,...,,,,,,,,,,
3,66,1211,Peters,7/20/1934,20,7,1934,1,0,,...,,,,,,,,,,
4,,2255,Bouts,10/29/1922,29,10,1922,1,0,2,...,171,29.6000003814697,21,19,right-handed,,,,,


# intersect stats and clinical using patient ID

In [85]:
dataFrameList = []

for i in range( dataFrameStats.shape[0]):
    temp = dataFrameClinical[   dataFrameClinical['patis'] == int(dataFrameStats['patient'].tolist()[i]) ]
    dataFrameList.append( temp )

result = pd.concat(dataFrameList)
result.head()

,idstudy,patis,name,Birthdte,birthday,birthmon,birthyea,gender,intake_who,nicotuse,...,height_fatandgrip,fat_perc_baseline,gripleft_baseline,gripright_baseline,prefside_hand,Date_meas_RT,weight_RT,fat_perc_RT,gripleft_RT,gripright_RT
299,341,43062,Buysch,6/26/1920,26,6,1920,1,1,1,...,,,,,,,,,,
318,805,43370,Moonen,4/23/1936,23,4,1936,1,1,1,...,,,,,,,,,,
319,,43392,Janssen,3/30/1936,30,3,1936,1,,,...,176,31.7999992370605,24,30,right-handed,2/28/2007,73,36.7999992370605,18,24
321,357,43402,Cuypers,1/7/1924,7,1,1924,1,1,2,...,,,,,,,,,,
322,358,43421,Peters,9/20/1930,20,9,1930,1,2,2,...,,,,,,,,,,


In [153]:
# clean up - only patient id and desired clinical endpoint
dataFrameIntermediate = pd.DataFrame()
dataFrameIntermediate['patient'] = result['patis'].tolist()
dataFrameIntermediate['surv2y'] = result['surv2y'].tolist()
#
# CAN ADD HERE
#
print dataFrameIntermediate.shape


# remove items that are empty ( or those that are only 0 or 1)
dataFrameIntermediate = dataFrameIntermediate [ ( dataFrameIntermediate['surv2y'] == '1' ) | (dataFrameIntermediate['surv2y'] == '0') ]
# remove duplicates - IMPORTANT
dataFrameIntermediate = dataFrameIntermediate.drop_duplicates('patient')

# unique patients with surv2y data
print dataFrameIntermediate.shape
dataFrameIntermediate.head(n=5)


(441, 2)
(248, 2)


,patient,surv2y
0,43062,0
1,43370,0
2,43392,0
3,43402,0
4,43421,0


# join by patient id and make noew column in stats

In [152]:
clinicalList = []
# loop through dataFrameStats and for each patient id, get clinical - 
# if it doesnt exist, put NaN
for i in range (dataFrameStats.shape[0]):
    # get id
    statPatientID  = dataFrameStats['patient'][i]
    # get entry
    temp = dataFrameIntermediate[ dataFrameIntermediate['patient'] == statPatientID  ]
    # nothing found
    if temp.shape[0] == 0:
        clinicalList.append('NaN')
    # one entry found - only one will be found because dataFrameIntermediate doesnt have duplicates
    else:
        clinicalList.append(temp['surv2y'].tolist()[0] )        

In [155]:
# now add list to frame
dataFrameStats['surv2y'] = clinicalList
print dataFrameStats.shape
dataFrameStats.head(n=5)

(537, 37)


,patient,nodule,Xspacing,Yspacing,Zspacing,Xmin,Ymin,Zmin,Xcount,Ycount,...,xPixelRange_G,yPixelRange_G,maskXmin_B,maskXmax_B,maskYmin_B,maskYmax_B,xPixelRange_B,yPixelRange_B,smoothed,surv2y
0,43062,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-166.0,512.0,512.0,...,96.0,72.0,129.0,225.0,175.0,247.0,96.0,72.0,True,0
1,43370,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-320.5,512.0,512.0,...,62.0,44.0,285.0,317.0,288.0,317.0,32.0,29.0,True,0
2,43392,0,0.976562,0.976562,3.0,-249.511719,-249.511719,-169.5,512.0,512.0,...,63.0,31.0,295.0,335.0,71.0,99.0,40.0,28.0,True,0
3,43402,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-136.0,512.0,512.0,...,71.0,68.0,171.0,217.0,74.0,142.0,46.0,68.0,True,0
4,43421,0,0.977000,0.977000,1.0,-249.623500,-249.623500,-227.5,512.0,512.0,...,59.0,66.0,275.0,335.0,181.0,247.0,60.0,66.0,True,0


In [159]:
# unique tumors with surv2y data
test = dataFrameStats [ ( dataFrameStats['surv2y'] == '1' ) | (dataFrameStats['surv2y'] == '0') ]
test.shape

(280, 37)

In [156]:
# save 
dataFrameStats.to_csv("D:/Dropbox/DFCI/PY-LUNG/output/Lung1_SliceStatsSurv2y.csv")